In [1]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=T4dKFeUEd6HOp7vCzavuI5zobB0AIcrGzEXzujJs"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2025-04-16T10:28:13+00:00","complexes":[{"id":"sr:complex:705","name":"Nacional","venues":[{"id":"sr:venue:70045","name":"Cancha Central","city_name":"Santiago","country_name":"Chile","country_code":"CHL","timezone":"America\/Santiago"}]},{"id":"sr:complex:1078","name":"Estadio de la Cartuja","venues":[{"id":"sr:venue:74856","name":"Centre Court","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"},{"id":"sr:venue:74858","name":"Court One","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"}]},{"id":"sr:complex:1495","name":"Sibur Arena","venues":[{"id":"sr:venue:1496","name":"COURT 1","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:1500","name":"CENTER COURT","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:62149","name":"Sibur Arena","city_na

In [2]:
#2. Transform nested JSON structures into a flat relational schema for analysis.
import json
import pandas as pd

data=json.loads(response.text)

# Flatten complexes data
def transform_complexes(data):
    complexes = []
    for comp in data['complexes']:
        complexes.append({
            'complex_id': comp['id'],
            'complex_name': comp['name']
        })
    return pd.DataFrame(complexes)

complexes_df=transform_complexes(data).drop_duplicates()

# Flatten categories data
def transform_venues(data):
    venues = []
    for comp in data['complexes']:
        for venue in comp.get('venues',[]):
            venues.append({
                'venue_id': venue['id'],
                'venue_name': venue['name'],
                'city_name': venue['city_name'],
                'country_name': venue['country_name'],
                'country_code': venue['country_code'],
                'timezone': venue['timezone'],
                'complex_id': comp['id']
            })
    return pd.DataFrame(venues)

venues_df=transform_venues(data)

print("Missing Values in Competitions:\n", complexes_df.isnull().sum())
print("Missing Values in Categories:\n", venues_df.isnull().sum())

complexes_df.to_csv("complexes.csv", index=False)
venues_df.to_csv("venues.csv", index=False)

Missing Values in Competitions:
 complex_id      0
complex_name    0
dtype: int64
Missing Values in Categories:
 venue_id        0
venue_name      0
city_name       0
country_name    0
country_code    0
timezone        0
complex_id      0
dtype: int64


In [3]:
#3. Create a SQL database with well-designed schema (e.g., defining appropriate data types and primary keys).
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password='',
    port=3307
)

cursor_db=mydb.cursor(buffered=True)

cursor_db.execute("CREATE DATABASE IF NOT EXISTS complex_db")

mydb_database="complex_db"

cursor_db.execute("USE complex_db")
cursor_db.execute("""
    CREATE TABLE IF NOT EXISTS Complexes (
        complex_id VARCHAR(50) PRIMARY KEY,
        complex_name VARCHAR(100) NOT NULL
    )
""")

cursor_db.execute("""
    CREATE TABLE IF NOT EXISTS Venues (
        venue_id VARCHAR(50) PRIMARY KEY,
        venue_name VARCHAR(100) NOT NULL,
        city_name VARCHAR(100) NOT NULL,
        country_name VARCHAR(100) NOT NULL,
        country_code CHAR(3) NOT NULL,
        timezone VARCHAR(100) NOT NULL,
        complex_id VARCHAR(50),
        FOREIGN KEY (complex_id) REFERENCES Complexes(complex_id)
    )
""")
# Apply Indexing for Faster Queries
try:
    cursor_db.execute("CREATE INDEX idx_complex ON Complexes(complex_id)")
    print("Index idx_complex created")
except mysql.connector.errors.ProgrammingError as e:
    if "Duplicate key name" in str(e):
        print("Index idx_complex already exists, skipping creation")
    else:
        raise  # Re-raise the exception if it's not a duplicate key error

try:
    cursor_db.execute("CREATE INDEX idx_venues ON Venues(complex_id)")
    print("Index idx_venues created")
except mysql.connector.errors.ProgrammingError as e:
    if "Duplicate key name" in str(e):
        print("Index idx_venues already exists, skipping creation")
    else:
        raise


# Verify Tables Exist
cursor_db.execute("SHOW TABLES")
tables = cursor_db.fetchall()
print("Tables Created:", tables)

# Commit Changes
mydb.commit()

# Load complexes data
complexes_df = pd.read_csv("complexes.csv")  # Assuming data was saved as CSV

# Insert into MySQL
for _, row in complexes_df.iterrows():
    cursor_db.execute(
        "INSERT INTO Complexes (complex_id, complex_name) VALUES (%s, %s) ON DUPLICATE KEY UPDATE complex_name = VALUES(complex_name)",
        (row['complex_id'], row['complex_name'])
    )

mydb.commit()
print("Complexes inserted successfully!")

#Insert Venues Data
venues_df = pd.read_csv("venues.csv")  # Load competition data

for col in venues_df.columns:
        if venues_df[col].dtype == 'object':
            venues_df[col] = venues_df[col].where(pd.notnull(venues_df[col]), None)



for _, row in venues_df.iterrows():
    cursor_db.execute(
        """INSERT INTO Venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
           VALUES (%s, %s, %s, %s, %s, %s, %s)
           ON DUPLICATE KEY UPDATE
               venue_name = VALUES(venue_name),
               city_name = VALUES(city_name),
               country_name = VALUES(country_name),
               country_code = VALUES(country_code),
               timezone = VALUES(timezone),
               complex_id = VALUES(complex_id)
        """,
        (
            row['venue_id'],
            row['venue_name'],
            row['city_name'],
            row['country_name'],
            row['country_code'],
            row['timezone'],
            row['complex_id']
        )
    )

mydb.commit()
print("Venues inserted successfully!")


Index idx_complex already exists, skipping creation
Index idx_venues already exists, skipping creation
Tables Created: [('complexes',), ('venues',)]
Complexes inserted successfully!
Venues inserted successfully!


In [4]:
#1. List all venues along with their associated complex name
from tabulate import tabulate
cursor_db.execute(""" SELECT v.venue_id,
                             v.venue_name,
                             c.complex_name
                      FROM Venues v
                      JOIN Complexes c ON v.complex_id = c.complex_id;""")
results_11 = cursor_db.fetchall()
print(tabulate(results_11, headers=[i[0] for i in cursor_db.description],  tablefmt='psql'))


+----------------+------------------------------------------------------------+------------------------------------------------------------+
| venue_id       | venue_name                                                 | complex_name                                               |
|----------------+------------------------------------------------------------+------------------------------------------------------------|
| sr:venue:66889 | Center Court                                               | Kindarena                                                  |
| sr:venue:66925 | Amelie Mauresmo                                            | Kindarena                                                  |
| sr:venue:66979 | Celine Dumerc                                              | Kindarena                                                  |
| sr:venue:74546 | Court 1                                                    | Kindarena                                                  |
| sr:venue:74

In [5]:
#2. Count the number of venues in each complex

cursor_db.execute(""" SELECT c.complex_name,
                             COUNT(v.venue_id) AS venue_count
                      FROM Complexes c
                      LEFT JOIN Venues v ON c.complex_id = v.complex_id
                      GROUP BY c.complex_id, c.complex_name;
""")
results_12 = cursor_db.fetchall()
print(tabulate(results_12, headers=[i[0] for i in cursor_db.description],  tablefmt='psql'))



+------------------------------------------------------------+---------------+
| complex_name                                               |   venue_count |
|------------------------------------------------------------+---------------|
| Kindarena                                                  |             4 |
| Estadio de la Cartuja                                      |             2 |
| Estadio German Becker                                      |             4 |
| Palais des sports de Gerland                               |             7 |
| Sibur Arena                                                |             4 |
| Palais Des Sport                                           |             7 |
| Palasport                                                  |             4 |
| Salle Steredenn                                            |             6 |
| Garanti Koza Arena                                         |             1 |
| Porsche-Arena                                     

In [6]:
#3. Get details of venues in a specific country (e.g., Chile)
cursor_db.execute("""SELECT * FROM Venues 
                     WHERE country_name = 'Chile';""")
results_13 = cursor_db.fetchall()
print(tabulate(results_13, headers=[i[0] for i in cursor_db.description],  tablefmt='psql'))

+----------------+-------------------------+--------------+----------------+----------------+------------------+------------------+
| venue_id       | venue_name              | city_name    | country_name   | country_code   | timezone         | complex_id       |
|----------------+-------------------------+--------------+----------------+----------------+------------------+------------------|
| sr:venue:13830 | Cancha 1                | Santiago     | Chile          | CHL            | America/Santiago | sr:complex:62419 |
| sr:venue:13832 | Cancha 3                | Santiago     | Chile          | CHL            | America/Santiago | sr:complex:62419 |
| sr:venue:15858 | Centre Court            | Santiago     | Chile          | CHL            | America/Santiago | sr:complex:62419 |
| sr:venue:15860 | Court 16                | Santiago     | Chile          | CHL            | America/Santiago | sr:complex:62419 |
| sr:venue:15862 | Court 15                | Santiago     | Chile          |

In [7]:
# 4. Identify all venues and their timezones
cursor_db.execute (""" SELECT venue_name, timezone, city_name
                       FROM venues;""")
results_14 = cursor_db.fetchall()
print(tabulate(results_14, headers=[i[0] for i in cursor_db.description],  tablefmt='psql'))

+------------------------------------------------------------+--------------------------------+----------------------------+
| venue_name                                                 | timezone                       | city_name                  |
|------------------------------------------------------------+--------------------------------+----------------------------|
| Estadio Monumental                                         | America/Lima                   | Lima                       |
| Stadium                                                    | Asia/Shanghai                  | Shenzhen                   |
| Court 2                                                    | Asia/Shanghai                  | Shenzhen                   |
| Court 7                                                    | Australia/Sydney               | Sydney                     |
| Court 20                                                   | Australia/Melbourne            | Melbourne                  |


In [8]:
# 5. Find complexes that have more than one venue
cursor_db.execute("""SELECT c.complex_name, count(v.venue_id) as Venue_Number
                     FROM complexes c
                     JOIN venues v ON c.complex_id=v.complex_id
                     GROUP BY c.complex_id, c.complex_name
                     Having count(v.venue_id) > 1;""")
results_15 = cursor_db.fetchall()
print(tabulate(results_15, headers=[i[0] for i in cursor_db.description],  tablefmt='psql'))


+------------------------------------------------------------+----------------+
| complex_name                                               |   Venue_Number |
|------------------------------------------------------------+----------------|
| Kindarena                                                  |              4 |
| Estadio de la Cartuja                                      |              2 |
| Estadio German Becker                                      |              4 |
| Palais des sports de Gerland                               |              7 |
| Sibur Arena                                                |              4 |
| Palais Des Sport                                           |              7 |
| Palasport                                                  |              4 |
| Salle Steredenn                                            |              6 |
| Porsche-Arena                                              |              3 |
| Rotterdam Ahoy                        

In [9]:
#6. List venues grouped by country
cursor_db.execute("""SELECT country_name, COUNT(*) AS venue_count
                     FROM Venues
                     GROUP BY country_name
                     ORDER BY venue_count DESC;
""")
results_16 = cursor_db.fetchall()
print(tabulate(results_16, headers=[i[0] for i in cursor_db.description],  tablefmt='psql'))


+----------------------+---------------+
| country_name         |   venue_count |
|----------------------+---------------|
| USA                  |           459 |
| Italy                |           245 |
| France               |           236 |
| China                |           207 |
| Spain                |           201 |
| Germany              |           130 |
| Mexico               |           120 |
| England              |           114 |
| Brazil               |           100 |
| Australia            |           100 |
| Japan                |            98 |
| Argentina            |            84 |
| Portugal             |            78 |
| Chile                |            65 |
| Canada               |            63 |
| Czechia              |            60 |
| Croatia              |            59 |
| Ecuador              |            54 |
| Turkiye              |            52 |
| Switzerland          |            49 |
| Colombia             |            45 |
| Austria       

In [10]:
# 7. Find all venues for a specific complex (e.g., Nacional)
cursor_db.execute("""SELECT v.*
FROM Venues v
JOIN Complexes c ON v.complex_id = c.complex_id
WHERE c.complex_name = 'Nacional';
""")
results_17 = cursor_db.fetchall()
print(tabulate(results_17, headers=[i[0] for i in cursor_db.description],  tablefmt='psql'))

+----------------+----------------+-------------+----------------+----------------+------------------+----------------+
| venue_id       | venue_name     | city_name   | country_name   | country_code   | timezone         | complex_id     |
|----------------+----------------+-------------+----------------+----------------+------------------+----------------|
| sr:venue:70045 | Cancha Central | Santiago    | Chile          | CHL            | America/Santiago | sr:complex:705 |
+----------------+----------------+-------------+----------------+----------------+------------------+----------------+
